In [95]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.stats import poisson
import penaltyblog as pb
from collections import Counter



In [96]:

df = pd.read_csv("veri_setleri/epl_data.csv")

#Tam adresi yazmak,dosyayı bulmuyorsa gereklidir.
#Örneğin:  df = pd.read_csv("/home/rdvn/Desktop/soccermatic/match_data.csv") 

In [97]:
df = df[df["season"]==2022]


In [98]:
df = df[df["comp"]=="Premier League"]

In [99]:
df.team.unique()

array(['Manchester City', 'Arsenal', 'Manchester United',
       'Newcastle United', 'Liverpool', 'Brighton and Hove Albion',
       'Aston Villa', 'Tottenham Hotspur', 'Brentford', 'Fulham',
       'Crystal Palace', 'Chelsea', 'Wolverhampton Wanderers',
       'West Ham United', 'Bournemouth', 'Nottingham Forest', 'Everton',
       'Leicester City', 'Leeds United', 'Southampton'], dtype=object)

In [100]:

d = {}
d1 = {}
d2 = {}
d3 = {}
for i,row in df.iterrows():
    team = row["team"]
    opp = row["opponent"]
    gf = row["gf"]
    ga = row["ga"]
    xg = row["xg"]
    xga = row["xga"]
    venue = row["venue"]
    sut = row["sot%"]
    if team not in d.keys():
        d[team]=team
    if opp not in d.keys():
        d[opp]=opp

    if gf not in d1.keys():
        d1[gf]=gf
    if ga not in d1.keys():
        d1[ga]=ga

    if venue == "Home":
        EvSahibi=d[team]
        DeplasmanTakimi=d[opp]
    elif venue=="Away":
          EvSahibi=d[opp]
          DeplasmanTakimi=d[team]

    if venue == "Home":
        EvGolleri=d1[gf]
        DeplasmanGolleri=d1[ga]
    elif venue=="Away":
          EvGolleri=d1[ga]
          DeplasmanGolleri=d1[gf]
    df.loc[i,'EvSahibi']=EvSahibi
    df.loc[i,'DeplasmanTakimi']=DeplasmanTakimi

    df.loc[i,'EvGolleri']=EvGolleri
    df.loc[i,'DeplasmanGolleri']=DeplasmanGolleri


In [101]:
df['EvSahibi'] = df['EvSahibi'].replace({"Huddersfield Town":"Huddersfield","West Bromwich Albion":"West Brom","Sheffield United":"Sheffield Utd","Nott'ham Forest":"Nottingham Forest",'West Ham United':'West Ham','Wolverhampton Wanderers': 'Wolves','Newcastle United':'Newcastle Utd','Tottenham Hotspur': 'Tottenham', 'Manchester United': 'Manchester Utd', 'Brighton and Hove Albion': 'Brighton'})
df['DeplasmanTakimi'] = df['DeplasmanTakimi'].replace({"Huddersfield Town":"Huddersfield","West Bromwich Albion":"West Brom","Sheffield United":"Sheffield Utd","Nott'ham Forest":"Nottingham Forest",'West Ham United':'West Ham','Wolverhampton Wanderers': 'Wolves','Newcastle United':'Newcastle Utd','Tottenham Hotspur': 'Tottenham', 'Manchester United': 'Manchester Utd', 'Brighton and Hove Albion': 'Brighton'})


In [102]:
df.drop_duplicates(subset=["EvSahibi","DeplasmanTakimi","season"],inplace=True)

In [103]:
df["round"] = df["round"].apply(lambda x: str(x).split()[1]).astype(int)
df["EvGolleri"] = df["EvGolleri"].astype(int)
df["DeplasmanGolleri"] = df["DeplasmanGolleri"].astype(int)



df.sort_values(by="date",inplace=True)

In [104]:

d = {}
d1 = {}
d2 = {}
d3 = {}
for i,row in df.iterrows():
    team = row["team"]
    opp = row["opponent"]
    gf = row["gf"]
    ga = row["ga"]
    xg = row["xg"]
    xga = row["xga"]
    venue = row["venue"]
    sut = row["sot%"]

 
    if team not in d.keys():
        d[team]=team
    if opp not in d.keys():
        d[opp]=opp

    
    if gf not in d1.keys():
        d1[gf]=gf
    if ga not in d1.keys():
        d1[ga]=ga


    if xg not in d2.keys():
        d2[xg]=xg
    if xga not in d2.keys():
        d2[xga]=xga	


    if venue == "Home":
        EvSahibi=d[team]
        DeplasmanTakimi=d[opp]

    elif venue=="Away":
          EvSahibi=d[opp]
          DeplasmanTakimi=d[team]

    if venue == "Home":
        home_xg=d2[xg]
        away_xg=d2[xga]

    elif venue=="Away":
        home_xg=d2[xga]
        away_xg=d2[xg]



    if venue == "Home":
        EvGolleri=d1[gf]
        DeplasmanGolleri=d1[ga]

    elif venue=="Away":
          EvGolleri=d1[ga]
          DeplasmanGolleri=d1[gf]



    df.loc[i,'EvSahibi']=EvSahibi
    df.loc[i,'DeplasmanTakimi']=DeplasmanTakimi
    
    df.loc[i,'EvGolleri']=EvGolleri
    df.loc[i,'DeplasmanGolleri']=DeplasmanGolleri

    df.loc[i,'home_xg']=home_xg
    df.loc[i,'away_xg']=away_xg


In [105]:
df['EvSahibi'] = df['EvSahibi'].replace({"West Bromwich Albion":"West Brom","Sheffield United":"Sheffield Utd","Nott'ham Forest":"Nottingham Forest",'West Ham United':'West Ham','Wolverhampton Wanderers': 'Wolves','Newcastle United':'Newcastle Utd','Tottenham Hotspur': 'Tottenham', 'Manchester United': 'Manchester Utd', 'Brighton and Hove Albion': 'Brighton'})
df['DeplasmanTakimi'] = df['DeplasmanTakimi'].replace({"Sheffield United":"Sheffield Utd","West Bromwich Albion":"West Brom","Nott'ham Forest":"Nottingham Forest",'West Ham United':'West Ham','Wolverhampton Wanderers': 'Wolves','Newcastle United':'Newcastle Utd','Tottenham Hotspur': 'Tottenham', 'Manchester United': 'Manchester Utd', 'Brighton and Hove Albion': 'Brighton'})


In [106]:
#   Sarta göre geriye G değerini döndürür.
#   Beraberlik veya tek farklı kazanılmışssa 1 ,2 farklı kazanılmıisa 1.5,
#   2'den fazla fark atılığ kazanılmışsa 2 dödürür. 
def G_degeri(anahtar):
    if anahtar == "acik":
        g = 1
    elif anahtar == "orta-acik":
        g = 1.5
    elif anahtar == "cok-acik":
        g = 2.0
   
 
    return g

#   İki takımın elo farkını alıp beklenen sonuç çıkartır.
#   57 EvSahibi avantajıdır. Beklenen sonuç iyi takım ve kötü takım
#   karşı karşıya geliyorsa yüksek olacaktır. Eğer kötü takım bu maçı
#   yenmeyi başarırsa hayli yüksek bir puan alacaktır.
def beklenen_sonuc(EvSahibi,DeplasmanTakimi):
    fark=EvSahibi-DeplasmanTakimi
    beklenen=(1/(10**(-(fark+57)/400)+1))
    return [np.round(beklenen,3),1-np.round(beklenen,3)]


#  Oynanan maçların gerçek sonucu. Kazanan 1,Kaybeden 0,Beraberlik 0.5
def gercek_sonuc(EvSahibi,DeplasmanTakimi):
    if EvSahibi>DeplasmanTakimi:
        EvSahibi=1
        DeplasmanTakimi=0
    elif EvSahibi<DeplasmanTakimi:
        EvSahibi=0
        DeplasmanTakimi=1
    elif EvSahibi==DeplasmanTakimi:
        EvSahibi=0.5
        DeplasmanTakimi=0.5
    return [EvSahibi,DeplasmanTakimi]


# Elo formülünün uygulandığı yer. Eğer takım yenilmişse,beklenene göre yüksek
# veya düşük olmak üzere ağırlıklar(K,G) negatif bir değerle çarpılır. Dolayısıyla elo puanına
# yeni puanın eklenmesine yerine farkı alınarak güncellenir.Söylediklerimin tam tersi de geçerlidir.
# K faktörü en küçük kareler ile bulunabilir lakin burda stoastik şekilde bulunmuştur.
def elo_hesapla(elo_EvSahibi,elo_DeplasmanTakimi,EvSahibi_goals,DeplasmanTakimi_goals,g_anahtar):
    K=40
    G =G_degeri(g_anahtar)
    ev_gercek,dep_gercek=gercek_sonuc(EvSahibi_goals,DeplasmanTakimi_goals)
    ev_beklenen,dep_beklenen=beklenen_sonuc(elo_EvSahibi,elo_DeplasmanTakimi)
    elo_EvSahibi_=elo_EvSahibi+K*G*(ev_gercek-ev_beklenen)
    elo_DeplasmanTakimi_=elo_DeplasmanTakimi+K*G*(dep_gercek-dep_beklenen)
    return elo_EvSahibi_,elo_DeplasmanTakimi_

#Temel eloyu depolayacak dict
anlik_elo={}

#   Her satırın verileriyle ayrı islen yapacağımız için
#   veri setinin satırları döngüye sokulur.
for index,satir in df.iterrows():
    EvSahibi=satir['EvSahibi']
    DeplasmanTakimi=satir['DeplasmanTakimi']
    EvSahibi_goals=satir['EvGolleri']
    DeplasmanTakimi_goals=satir['DeplasmanGolleri']
    
    #G değerini ayarlamak için
    if EvGolleri - DeplasmanGolleri == 0:
        anahtar ="acik"
    elif EvGolleri - DeplasmanGolleri == 1:
        anahtar ="acik"
    elif EvGolleri - DeplasmanGolleri == 2:
        anahtar ="orta-acik"
    elif EvGolleri - DeplasmanGolleri > 2:
        anahtar ="cok-acik"
   
    elif DeplasmanGolleri - EvGolleri == 0:
        anahtar ="acik"
    elif DeplasmanGolleri - EvGolleri ==1:
        anahtar ="acik"
    elif DeplasmanGolleri - EvGolleri == 2:
        anahtar ="acik"
    elif DeplasmanGolleri - EvGolleri > 2:
        anahtar ="cok-acik"


    #Temel elo 1500'den başlıyor.
    if EvSahibi not in anlik_elo.keys():
        anlik_elo[EvSahibi]=1500
    if DeplasmanTakimi not in anlik_elo.keys():
        anlik_elo[DeplasmanTakimi]=1500

        
    elo_EvSahibi_onceki=anlik_elo[EvSahibi]
    elo_DeplasmanTakimi_onceki=anlik_elo[DeplasmanTakimi]
    elo_EvSahibi_sonraki,elo_DeplasmanTakimi_sonraki=elo_hesapla(elo_EvSahibi_onceki,elo_DeplasmanTakimi_onceki,EvSahibi_goals,DeplasmanTakimi_goals,anahtar)
    anlik_elo[EvSahibi]=elo_EvSahibi_sonraki
    anlik_elo[DeplasmanTakimi]=elo_DeplasmanTakimi_sonraki
    
    
    
    df.loc[index,'Elo_ev_sonraki']=elo_EvSahibi_sonraki
    df.loc[index,'Elo_dep_sonraki']=elo_DeplasmanTakimi_sonraki 
    df.loc[index,'Elo_ev_onceki']=elo_EvSahibi_onceki
    df.loc[index,'Elo_dep_onceki']=elo_DeplasmanTakimi_onceki

In [107]:
df

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,EvSahibi,DeplasmanTakimi,EvGolleri,DeplasmanGolleri,home_xg,away_xg,Elo_ev_sonraki,Elo_dep_sonraki,Elo_ev_onceki,Elo_dep_onceki
37,0,2022-08-05,20:00,Premier League,1,Fri,Away,W,2.0,0.0,...,Crystal Palace,Arsenal,0,2,1.2,1.0,1453.52,1546.48,1500.00,1500.00
407,0,2022-08-06,17:30,Premier League,1,Sat,Away,W,1.0,0.0,...,Everton,Chelsea,0,1,0.7,1.5,1453.52,1546.48,1500.00,1500.00
222,0,2022-08-06,15:00,Premier League,1,Sat,Away,L,0.0,2.0,...,Bournemouth,Aston Villa,2,0,0.6,0.7,1533.52,1466.48,1500.00,1500.00
444,0,2022-08-06,15:00,Premier League,1,Sat,Away,L,1.0,2.0,...,Leeds United,Wolves,2,1,0.8,1.3,1533.52,1466.48,1500.00,1500.00
111,0,2022-08-06,15:00,Premier League,1,Sat,Home,W,2.0,0.0,...,Newcastle Utd,Nottingham Forest,2,0,1.7,0.3,1533.52,1466.48,1500.00,1500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,43,2023-05-21,14:00,Premier League,37,Sun,Home,W,3.0,1.0,...,Brighton,Southampton,3,1,2.5,0.6,1561.76,1240.32,1552.88,1249.20
35,56,2023-05-21,16:00,Premier League,37,Sun,Home,W,1.0,0.0,...,Manchester City,Chelsea,1,0,1.2,1.2,1825.12,1375.60,1820.80,1379.92
147,44,2023-05-22,20:00,Premier League,37,Mon,Home,D,0.0,0.0,...,Newcastle Utd,Leicester City,0,0,2.1,0.1,1616.96,1349.20,1649.28,1316.88
36,57,2023-05-24,20:00,Premier League,32,Wed,Away,D,1.0,1.0,...,Brighton,Manchester City,1,1,2.2,1.8,1583.04,1803.84,1561.76,1825.12


In [108]:
#Temel eloyu depolayacak dict
anlik_elo={}

#   Her satırın verileriyle ayrı islen yapacağımız için
#   veri setinin satırları döngüye sokulur.
for index,satir in df.iterrows():
    EvSahibi=satir['EvSahibi']
    DeplasmanTakimi=satir['DeplasmanTakimi']
    EvSahibi_goals=satir['EvGolleri']
    DeplasmanTakimi_goals=satir['DeplasmanGolleri']
    
    #G değerini ayarlamak için
    if EvGolleri - DeplasmanGolleri == 0:
        anahtar ="acik"
    elif EvGolleri - DeplasmanGolleri == 1:
        anahtar ="acik"
    elif EvGolleri - DeplasmanGolleri == 2:
        anahtar ="orta-acik"
    elif EvGolleri - DeplasmanGolleri > 2:
        anahtar ="cok-acik"
   
    elif DeplasmanGolleri - EvGolleri == 0:
        anahtar ="acik"
    elif DeplasmanGolleri - EvGolleri ==1:
        anahtar ="acik"
    elif DeplasmanGolleri - EvGolleri == 2:
        anahtar ="acik"
    elif DeplasmanGolleri - EvGolleri > 2:
        anahtar ="cok-acik"


    #Temel elo 1500'den başlıyor.
    if EvSahibi not in anlik_elo.keys():
        anlik_elo[EvSahibi]=1500
    if DeplasmanTakimi not in anlik_elo.keys():
        anlik_elo[DeplasmanTakimi]=1500

        
    elo_EvSahibi_onceki=anlik_elo[EvSahibi]
    elo_DeplasmanTakimi_onceki=anlik_elo[DeplasmanTakimi]
    elo_EvSahibi_sonraki,elo_DeplasmanTakimi_sonraki=elo_hesapla(elo_EvSahibi_onceki,elo_DeplasmanTakimi_onceki,EvSahibi_goals,DeplasmanTakimi_goals,anahtar)
    anlik_elo[EvSahibi]=elo_EvSahibi_sonraki
    anlik_elo[DeplasmanTakimi]=elo_DeplasmanTakimi_sonraki
    
    
    
    df.loc[index,'Elo_ev_sonraki']=elo_EvSahibi_sonraki
    df.loc[index,'Elo_dep_sonraki']=elo_DeplasmanTakimi_sonraki 
    df.loc[index,'Elo_ev_onceki']=elo_EvSahibi_onceki
    df.loc[index,'Elo_dep_onceki']=elo_DeplasmanTakimi_onceki

In [109]:


for i,row in df.iterrows():
    home = row["home_xg"]
    away = row["away_xg"]
    elo_h = row["Elo_ev_onceki"]
    elo_a = row["Elo_dep_onceki"]
    ev_gol = row["EvGolleri"]
    dep_gol = row["DeplasmanGolleri"]
  

    ev_xg = (elo_a/elo_h) *home  
    dep_xg = (elo_h/elo_a) *away 
    
    
    a = ((ev_gol - ev_xg))
    b = ((dep_gol -dep_xg))




    df.at[i,"weight_h"] = round(ev_xg)
    df.at[i,"weight_a"] = round(dep_xg)




    

    


    
    
  


In [110]:
df1 = df.rename(columns={"weight_h":"EloxG(Ev)","round":"Hafta","weight_a":"EloxG(Deplasman)"}).copy()              


In [111]:
df=df[["season","round","sot%","date","EvSahibi","EvGolleri","DeplasmanGolleri","DeplasmanTakimi","home_xg","away_xg",'Elo_ev_sonraki', 'Elo_dep_sonraki', 'Elo_ev_onceki', 'Elo_dep_onceki',"weight_h","weight_a"]]
df = df.rename(columns={"season":"Sezon","round":"Maç Haftası","sot%":'SutYuzdesi',"date":"Tarih","EvGolleri":"Ev Gol","DeplasmanGolleri":"Deplasman Gol","EvSahibi":"EvSahibi","DeplasmanTakimi":"Deplasman","home_xg":"Ev xG","away_xg":"Deplasman xG","Elo_ev_sonraki":"Ev Sonraki Elo","Elo_dep_sonraki":"Deplasman Sonraki Elo","Elo_ev_önceki":"Ev Önceki Elo","Elo_dep_önceki":"Deplasman Önceki Elo","weight_h":"EvYenixG","weight_a":"DepYenixG"})

In [112]:
df2 = df.copy()
df2.to_csv("elo_xg.csv")

In [113]:
df[["Tarih"]] = df[["Tarih"]].apply(pd.to_datetime)

In [114]:
ekle = []
for idx, row in df.iterrows():
        EvGolleri = row["Ev Gol"]   
        DeplasmanGolleri = row["Deplasman Gol"]   
        
        if EvGolleri > DeplasmanGolleri :
           ekle.append("E")
        elif  EvGolleri == DeplasmanGolleri:
           ekle.append("B")
        elif EvGolleri < DeplasmanGolleri:
           ekle.append("D")

df["MS"] = ekle

In [115]:
# # xi_vals = [0.0, 0.0002, 0.0004, 0.0006, 0.0008, 0.001, 0.0012, 0.0014, 0.0016, 0.0018, 
# #             0.002, 0.0025, 0.003, 0.0035,  0.0035, 0.004,  0.0045, 0.005]

# xi_vals = [0.0, 0.0002, 0.0004, 0.0006, 0.0008, 0.001, 0.0012, 0.0014, 0.0016, 0.0018,
#              0.002, 0.0025, 0.003, 0.0035,  0.0035, 0.004,  0.0045,0.005,0.0055,0.006,0.0065,0.0070]

In [116]:
# plt.plot(xi_vals, rps)
# plt.ticklabel_format(useOffset=False)
# plt.xlabel("ξ")
# plt.ylabel("RPS")

In [117]:
import penaltyblog as pb
import pandas as pd
import numpy as np



def calculate_mean_rps(dc, df):
    rps = list()
    for idx, row in df.iterrows():
        if row["MS"] == "B":
            outcome = 0
        elif row["MS"] == "E":
            outcome = 1
        elif row["MS"] == "D":
            outcome = 2

        predictions = dc.predict(row["EvSahibi"], row["Deplasman"]).home_draw_away

        rps.append(pb.metrics.rps(predictions, outcome))

    return np.mean(rps)

# xi_degereleri = [0.0, 0.0002, 0.0004, 0.0006, 0.0008, 0.001, 0.0012, 0.0014, 0.0016, 0.0018,
#              0.002, 0.0025, 0.003, 0.0035,  0.0035, 0.004,  0.0045,0.005,0.0065]
xi_degereleri = [0.0014]


rps = list()

for xi in xi_degereleri:
    #Test için istediğin kadar veriyi burdan al
    # test = df.iloc[1730:]

    #Train verisi.
    # df_ = df[:1730]

    df_ = df.copy()
    df_["weight"] = pb.models.dixon_coles_weights(df_["Tarih"], xi)
    dc = pb.models.DixonColesGoalModel(
        df_["EvYenixG"],
        df_["DepYenixG"],
        df_["EvSahibi"],
        df_["Deplasman"],
        df_["weight"]
        )
    dc.fit()    

#Farklı xi değerleri için RPS bakılacaksa burayı aç.
#     rps.append(calculate_mean_rps(dc, test))
# print(rps)


####DİREK BELLİ BİR xi için çalışsacaksa burayı kullan
xi_degereleri = [0.0014]


     fun: 771.4315047331229
     jac: array([-0.00159454, -0.00162506, -0.00093842, -0.0035553 ,  0.01054382,
       -0.00473022, -0.0017395 , -0.00618744, -0.00257874, -0.00497437,
       -0.00331116, -0.00292969, -0.00570679, -0.00424194, -0.00238037,
        0.0001297 ,  0.00074768, -0.00240326, -0.00228119, -0.00024414,
       -0.00312805, -0.00326538,  0.0010376 , -0.00172424, -0.00218201,
       -0.00311279, -0.00213623, -0.00086212,  0.00022125, -0.00144958,
       -0.00097656, -0.00330353,  0.0012207 , -0.00248718, -0.00286865,
       -0.0015564 , -0.00431061, -0.00215912, -0.00283051, -0.00413513,
       -0.04187012, -0.0075531 ])
 message: 'Optimization terminated successfully'
    nfev: 805
     nit: 18
    njev: 18
  status: 0
 success: True
       x: array([ 1.20741725,  0.88938185,  0.7988752 ,  1.07822415,  1.36646003,
        0.89980614,  0.65476577,  0.9366427 ,  0.85751953,  1.02926628,
        1.07566423,  1.25424387,  1.2899609 ,  1.18574859,  1.23880745,
        0.

In [118]:
df['EvYenixG'] = df['EvYenixG'].round()
df['DepYenixG'] = df['DepYenixG'].round()

In [119]:
# def rho_ayarlayici(ev_gol,dep_gol,beklenen_ev, beklenen_dep, rho):
#     if ev_gol == 0 and dep_gol == 0:
#         return 1 - (beklenen_ev * beklenen_dep * rho)
#     elif ev_gol == 0 and dep_gol == 1:
#         return 1 + (beklenen_ev * rho)
#     elif ev_gol == 1 and dep_gol == 0:
#         return 1 + (beklenen_dep * rho)
#     elif ev_gol == 1 and dep_gol == 1:
#         return 1-rho
#     else:
#         return 1

In [120]:
# def log_olabilirlik(
 
#     ev_xg,
#     dep_xg,   
#     EvHucumGucu,
#     EvSavunmaGucu,
#     DeplasmanAtakGucu,
#     DeplasmanSavunmaGucu,
#     ev_avantaji,
#     rho,
#     agirlik,
# ):
#     # İki takımın atak ve savunma gücünü belirleyecek fonksiyonlar
#     ev_beklenen = np.exp(EvHucumGucu + DeplasmanSavunmaGucu +ev_avantaji) 
#     dep_beklenen = np.exp(DeplasmanAtakGucu + EvSavunmaGucu) 
    
#     # Modelin gercek xG değerini verme olasılıkları. 
#     ev_olabilirlik = poisson.pmf(ev_xg, ev_beklenen)
#     deplasman_olabilirlik = poisson.pmf(dep_xg, dep_beklenen)
   
#     rho_eklemesi= rho_ayarlayici(
        
#         ev_xg,
#         dep_xg, 
#         ev_beklenen ,
#         dep_beklenen,
#         rho,
#     )

#     # Poisson dağılımı yalnızca 0'dan büyük değerler için çalışır.
#     # Zatben tahmin edilmesi istenen xG pozitif  bir değer
#     # olduğuundan dolayı negatif beklenen gelirse diye uyarıcı konuldu.
#     if ev_gol < 0 or dep_gol < 0 or rho_eklemesi<0:
#         return 10000    



#     # Değer log ile dönüştürülerek, bilgisayarın hesap yapabileceği zemine getirildi. 
#     log_olabilirlik = (np.log(ev_olabilirlik) + np.log(deplasman_olabilirlik) +np.log(rho_eklemesi)) *agirlik

#     # Makismumu bulmak yerine minumum bulmak istendiği için, değerler negatif alındı.
#     return -log_olabilirlik  


In [121]:
# def agirlik_fonksiyonu(xi, yuzde):
#     #Ağırlık fonksiyonu.
#     return np.exp(-xi * yuzde)

In [122]:

# def poisson_modeli(df_,xi=0.004):
#     #   her takım için ayrı hücum ve savunma gücü ile birlikte 
#     #   beklenilen gol fonksiyonu oluşturacağımız için takım isimlerini
#     #   beliritici olarak alındı. Arsenal   hucum Gücü: 1.028 gibi.
#     takimlar = np.sort(np.unique(np.concatenate([df_["EvSahibi"], df_["Deplasman"]])))
#     #  Takımın hücum veya savunma gücü bulunmaya başlanınca ortaya
#     #  belki de binlerce parametre çıkacaktır. Yeni katsayıları bir yerde durdurmak gerektiği için
#     #  orjinal makalede de belirtildiği üzere hücum gücunu takım başına model

#     df_["gecen_gun"] = (df_["Tarih"].max() - df_["Tarih"]).dt.days
#     df_["agirlik"] = agirlik_fonksiyonu(xi, df_["gecen_gun"])

#     takim_sayisi = len(takimlar)
#     parametreler = np.concatenate(
#         (
#             np.random.uniform(0.5, 1.5, (takim_sayisi)),  #Atak Gücü
#             np.random.uniform(0, -1, (takim_sayisi)),  #Savunma Gücü
#             [0.25],
#             [-0.1],
#         )
#     )
 
#     def _fit(params, df_, teams):
#         hucum_parametreleri = dict(zip(teams, params[:takim_sayisi]))
#         savunma_parametreleri = dict(zip(teams, params[takim_sayisi : (2 * takim_sayisi)]))
#         ev_avantaji = params[-2]
#         rho = params[-1]

#         llk = list()
#         for idx, satir in df_.iterrows():
#             log_ = log_olabilirlik(
#                 satir["EvYenixG"],
#                 satir["DepYenixG"],
#                 hucum_parametreleri[satir["EvSahibi"]],
#                 savunma_parametreleri[satir["EvSahibi"]],
#                 hucum_parametreleri[satir["Deplasman"]],
#                 savunma_parametreleri[satir["Deplasman"]],
#                 ev_avantaji,
#                 rho,
#                 satir["agirlik"],
#             )
#             llk.append(log_)
#         # Tüm takımların katıldığı toplam olasıklar döndürüldü.
#         return np.sum(llk)
      

#     # 100 denemeden sonra optimal katsayı bulunmazsa durması
#     # gerektiği burda belirtildi.Çıktıları göstermemesi istendi.
#     ayarlama = {
#         "maxiter": 100,
#         "disp": False,
#     }
#     #  Takim sayısını alırken anlatılan olay burdda gerçekleştirilir.
#     #  Hücum gücü toplamı takım sayısına eşit olunca optimizasyon duracaktır.    
#     durma_noktasi = [{"type": "eq", "fun": lambda x: (sum(x[:takim_sayisi])) - takim_sayisi}]



#     # Optimizasyon burda çalışır.
   
#     hata_kucultme = minimize(
#         _fit,
#         parametreler,
#         args=(df_, takimlar),
#         constraints=durma_noktasi,
#         options=ayarlama,
#     )

#     model_parametreleri = dict(
#         zip(
#             ["hucum_" + takim for takim in takimlar]
#             + ["savunma_" + takim for takim in takimlar]
#             +["ev_avantaji","rho"]
#            ,hata_kucultme["x"]
#         )
#     )
#     print("log-likelihood",hata_kucultme["fun"])
#     return model_parametreleri 



In [123]:
def tahmin(parametre, ev_sahibi, deplasman):
    ev_atak_gucu = parametre["hucum_" + ev_sahibi]
    ev_savunma_gucu = parametre["savunma_" + ev_sahibi]
    dep_atak_gucu = parametre["hucum_" + deplasman]
    dep_savunma_gucu = parametre["savunma_" + deplasman]
    ev_avantaji =  parametre["ev_avantaji"]
    rho =   parametre["rho"]

    ev_beklenen = np.exp(ev_atak_gucu + dep_savunma_gucu )
    deplasman_beklenen = np.exp(dep_atak_gucu + ev_savunma_gucu)

    ev_olasiklar = poisson.pmf(range(10), ev_beklenen)
    dep_olasiliklar = poisson.pmf(range(10), deplasman_beklenen)

    matris = np.outer(ev_olasiklar, dep_olasiliklar)
  
    matris[0, 0] *= 1 - ev_beklenen * deplasman_beklenen * rho
    matris[0, 1] *= 1 + ev_beklenen * rho
    matris[1, 0] *= 1 + deplasman_beklenen * rho
    matris[1, 1] *= 1 - rho    
    
    

    # ev_gol,dep_gol = gol_bulucu(matris)
   
    


    return ev_beklenen,deplasman_beklenen

In [124]:
def tahmin2(parametre, ev_sahibi, deplasman):
    ev_atak_gucu = parametre["hucum_" + ev_sahibi]
    ev_savunma_gucu = parametre["savunma_" + ev_sahibi]
    dep_atak_gucu = parametre["hucum_" + deplasman]
    dep_savunma_gucu = parametre["savunma_" + deplasman]
    ev_avantaji =  parametre["ev_avantaji"]
    rho =   parametre["rho"]

    ev_beklenen = np.exp(ev_atak_gucu + dep_savunma_gucu +ev_avantaji)
    deplasman_beklenen = np.exp(dep_atak_gucu + ev_savunma_gucu)

    ev_olasiklar = poisson.pmf(range(10), ev_beklenen)
    dep_olasiliklar = poisson.pmf(range(10), deplasman_beklenen)
 

    matris = np.outer(ev_olasiklar, dep_olasiliklar)

    matris[0, 0] *= 1 - ev_beklenen * deplasman_beklenen * rho
    matris[0, 1] *= 1 + ev_beklenen * rho
    matris[1, 0] *= 1 + deplasman_beklenen * rho
    matris[1, 1] *= 1 - rho    
    

    

    ev = np.sum(np.tril(matris, -1))
    beraberlik = np.sum(np.diag(matris))
    dep = np.sum(np.triu(matris, 1))


    return ev,beraberlik,dep

In [125]:

for i, satir in df.iterrows():
    ev = satir["EvSahibi"]
    dep = satir["Deplasman"]
    # tahminler =tahmin(model,ev,dep)
    tahminler = dc.predict(ev,dep)

    # ind = np.unravel_index(tahminler.argmax(), tahminler.shape)
    df.at[i,"Ev Tahmin Edilen Gol"] =tahminler.home_goal_expectation
    df.at[i,"Deplasman Tahmin Edilen Gol"] =tahminler.away_goal_expectation
for i, satir in df.iterrows():
    ev = satir["EvSahibi"]
    dep = satir["Deplasman"]
    # tahminler2 = tahmin2(model,ev,dep)
    tahminler2 = dc.predict(ev,dep)

    df.at[i,"MS EV"] = tahminler2.home_win
    df.at[i,"MS BERABERLIK"] = tahminler2.draw
    df.at[i,"MS DEPLASMAN"] = tahminler2.away_win
    df.at[i,"KG Var"] = tahminler2.both_teams_to_score
    df.at[i,"2.5 Üstü"] = tahminler2.total_goals("over",2.5)
    


In [126]:

puan_hesapla = df[['Tarih','EvSahibi','Deplasman','Ev Tahmin Edilen Gol','Deplasman Tahmin Edilen Gol']]

puan = puan_hesapla["Ev Tahmin Edilen Gol"] - puan_hesapla["Deplasman Tahmin Edilen Gol"]
ev_puani = [3 if skor>0 else 1 if skor == 0 else 0 for skor in puan]
deplasman_puani = [3 if skor < 0 else 1 if skor == 0 else 0 for skor in puan]

puan_hesapla = puan_hesapla.assign(ev_puani = ev_puani,deplasman_puani = deplasman_puani)

puan_hesapla.rename(columns={"EvSahibi":"EvSahibi"},inplace=True)
def toplami_al(takim_ismi,toplam=None,ev_toplam=None,deplasman_toplam=None) -> int:
    

    ev_puani = puan_hesapla.query(f"EvSahibi=='{takim_ismi}'")["ev_puani"]
    deplasman_puani = puan_hesapla.query(f"Deplasman=='{takim_ismi}'")["deplasman_puani"]
    toplam_= np.sum(deplasman_puani) + np.sum(ev_puani)

    if toplam:
         return toplam_
    if ev_toplam:
         return np.sum(ev_puani)

    if deplasman_toplam:
         return np.sum(deplasman_puani)

takim_isimleri = puan_hesapla["EvSahibi"].unique()
toplam_puan = {}
toplam_ev_puani={}
toplam_deplasman_puani={}
for index,takim in enumerate(takim_isimleri):
     toplam_puan[takim]=toplami_al(takim,toplam=1)
     toplam_ev_puani[takim]=toplami_al(takim,ev_toplam=1)
     toplam_deplasman_puani[takim]=toplami_al(takim,deplasman_toplam=1)


def get_total_goals(takim_ismi=str,switch=str) -> int :

    if switch == "ev":
        gol_ev = puan_hesapla.query(f"EvSahibi=='{takim_ismi}'")["Ev Tahmin Edilen Gol"]
        return np.sum(gol_ev)
    elif switch =="dep":
        gol_dep = puan_hesapla.query(f"Deplasman=='{takim_ismi}'")["Deplasman Tahmin Edilen Gol"]
        return np.sum(gol_dep)
    

def get_total_goals1(takim_ismi=str,switch=str) -> int :

    if switch == "ev":
        yenilen_ev = puan_hesapla.query(f"EvSahibi=='{takim_ismi}'")["Deplasman Tahmin Edilen Gol"]
        return np.sum(yenilen_ev)
    elif switch =="dep":
        yenilen_dep = puan_hesapla.query(f"Deplasman=='{takim_ismi}'")["Ev Tahmin Edilen Gol"]
        return np.sum(yenilen_dep)



toplam_gol_ev = {}
toplam_gol_dep = {}
toplam_gol = {}
for index,takim in enumerate(takim_isimleri):
     toplam_gol_ev[takim]=get_total_goals(takim,"ev")
     toplam_gol_dep[takim]=get_total_goals(takim,"dep")

toplam_gol_ev_ = {}
toplam_gol_dep_ = {}
toplam_gol_ = {}
for index,takim in enumerate(takim_isimleri):
     toplam_gol_ev_[takim]=get_total_goals1(takim,"ev")
     toplam_gol_dep_[takim]=get_total_goals1(takim,"dep")

tablo = pd.DataFrame({"Toplam Puan":toplam_puan,"Ev":toplam_gol_ev ,"Dep":toplam_gol_dep,"Yenilen Ev":toplam_gol_ev_,"Yenilen Dep":toplam_gol_dep_
                        })
tablo = tablo.sort_values('Toplam Puan', ascending=False)

In [127]:
tablo["Atılan Gol"] = tablo.Ev + tablo.Dep
tablo["Yenilen Gol"] =tablo["Yenilen Ev"] + tablo["Yenilen Dep"]
tablo["Oynanan"] = 38


In [128]:
df["toplam_gol"] = df["Ev Gol"] + df["Deplasman Gol"] 
df["toplam_gol_model"]=df["Ev Tahmin Edilen Gol"] + df["Deplasman Tahmin Edilen Gol"] 

In [129]:
df[["Ev Gol","Deplasman Gol","Ev Tahmin Edilen Gol","Deplasman Tahmin Edilen Gol"]] = df[["Ev Gol","Deplasman Gol","Ev Tahmin Edilen Gol","Deplasman Tahmin Edilen Gol"]].astype(int)
for i,satir in df.iterrows():
    ev_gol = satir["Ev Gol"]
    dep_gol = satir["Deplasman Gol"]
    ev_tahmin = satir["MS EV"]
    dep_tahmin = satir["MS DEPLASMAN"]
    b_tahmin = satir["MS BERABERLIK"]

    ev_tahmin_ = satir["Ev Tahmin Edilen Gol"]
    dep_tahmin_ = satir["Deplasman Tahmin Edilen Gol"]
    
    
    t = (ev_tahmin,dep_tahmin,b_tahmin)
    
    o1 = b_tahmin - ev_tahmin
    o2 = b_tahmin - dep_tahmin
    o1=np.ceil(o1)
    o2=np.ceil(o2)

    if   b_tahmin == max(t):
            df.at[i,"Tahmin Edilen MS"] = "Beraberlik"
            if ev_gol == dep_gol:
                 df.at[i,"Tahmin Edilen Sonuç"] = 1  
            else:
                df.at[i,"Tahmin Edilen Sonuç"] = 0
           
    
    elif ev_tahmin == max(t):
        df.at[i,"Tahmin Edilen MS"] = "Ev"
        if ev_gol>dep_gol:
            df.at[i,"Tahmin Edilen Sonuç"] = 1
        else:
            df.at[i,"Tahmin Edilen Sonuç"] = 0   
    elif dep_tahmin ==max(t):
        df.at[i,"Tahmin Edilen MS"] = "Deplasman"
        if dep_gol > ev_gol:
            df.at[i,"Tahmin Edilen Sonuç"] = 1 
        else:
            df.at[i,"Tahmin Edilen Sonuç"] = 0    
    

            


    if ev_gol>dep_gol:
        df.at[i,"Gerçek Durum"] = "Ev"
    elif ev_gol < dep_gol:
        df.at[i,"Gerçek Durum"] = "Deplasman"
    elif ev_gol == dep_gol:
        df.at[i,"Gerçek Durum"] = "Beraberlik"

    if ev_gol + dep_gol > 2:
        df.at[i,"Gerçek 2.5 Üstü"] = 1
    else:
        df.at[i,"Gerçek 2.5 Üstü"] = 0
    if ev_gol + dep_gol < 3:
        df.at[i,"Gerçek 2.5 Altı"] = 1
    else:
        df.at[i,"Gerçek 2.5 Altı"] = 0
    
    if ev_tahmin_ + dep_tahmin_ > 2:
        df.at[i,"Tahmin 2.5 Üstü"] = 1
    else:
        df.at[i,"Tahmin 2.5 Üstü"] = 0
    if ev_tahmin_ + dep_tahmin_ < 3:
        df.at[i,"Tahmin 2.5 Altı"] = 1
    else:
        df.at[i,"Tahmin 2.5 Altı"] = 0
    

    if bool(ev_gol == ev_tahmin_) & bool(dep_gol==dep_tahmin_):
        df.at[i,"Doğru Skor"] = 1
    else:
        df.at[i,"Doğru Skor"] = 0


df["Tahmin 2.5 Üstü"].fillna(1,inplace=True)

In [130]:
kazanılan = {}
berabere = {}
kaybedilen = {}
kazanilan_ev = Counter()
berabere = Counter()
kazanilan_dep = Counter()
kaybedilen_dep = Counter()
kaybedilen_ev = Counter()
for i,row in df.iterrows():
    ev = row["EvSahibi"] 
    dep = row["Deplasman"] 
    sonuc = row["Tahmin Edilen MS"]

    if sonuc == "Ev":
       kazanilan_ev[ev] += 1
       kaybedilen_dep[dep] +=1
    elif sonuc == "Deplasman":
      kazanilan_dep[dep] +=1
      kaybedilen_ev[ev] +=1
    elif sonuc =="Beraberlik":
       berabere[ev] +=1
       berabere[dep] +=1

     

tablo["EvMaç"] = kazanilan_ev
tablo["DepMaç"] = kazanilan_dep
tablo["EvMaçK"] = kaybedilen_ev
tablo["DepMaçK"] = kaybedilen_dep
tablo["beraber"] = berabere



tablo["Kazanılan Maç"] = tablo["EvMaç"] + tablo["DepMaç"]
tablo["Kaybedilen Maç"] = tablo["EvMaçK"] + tablo["DepMaçK"]
tablo["Beraberlik"] = tablo["beraber"]

In [131]:
tablo = tablo[["Oynanan","Toplam Puan","Kazanılan Maç","Kaybedilen Maç","Beraberlik","Atılan Gol","Yenilen Gol"]]
tablo.fillna(2,inplace=True)
tablo[tablo.columns] = tablo[tablo.columns].astype(int)

In [132]:
tablo["Toplam Puan"] = np.abs(tablo["Kazanılan Maç"]*3  + tablo["Beraberlik"])

In [133]:
tablo.sort_values(by="Toplam Puan",ascending=False,inplace=True)


In [134]:
df[["Doğru Skor","Gerçek 2.5 Üstü","Gerçek 2.5 Altı","Tahmin 2.5 Üstü","Tahmin 2.5 Altı"]] = df[["Doğru Skor","Gerçek 2.5 Üstü","Gerçek 2.5 Altı","Tahmin 2.5 Üstü","Tahmin 2.5 Altı"]].astype(int)

In [135]:
# plt.figure(figsize = (15,8))
# sns.lineplot(x='Tarih', y='Elo_ev_onceki', hue='EvSahibi', ci = None,
#    data=df[(df['EvSahibi'].isin([  
#        'Newcastle Utd'
       
#         ]))])
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [136]:
df = df[["Tarih","Maç Haftası","EvSahibi","Deplasman","Ev Gol","Deplasman Gol","Ev Tahmin Edilen Gol","Deplasman Tahmin Edilen Gol","Doğru Skor","Tahmin Edilen Sonuç","Gerçek Durum","Tahmin Edilen MS", 'MS EV', 'MS BERABERLIK', 'MS DEPLASMAN',"2.5 Üstü","KG Var"]]

In [137]:



bet = pd.read_csv("/home/rdvn/Downloads/veri_setleri/p_data/E0 (3).csv")
bet = bet[["HomeTeam","AwayTeam","B365H","B365D","B365A"]]
bet.rename(columns={"HomeTeam":"EvSahibi","AwayTeam":"Deplasman"},inplace=True)
bet['EvSahibi'] = bet['EvSahibi'].replace({'Leeds':'Leeds United','Man United':'Manchester Utd','Leicester':'Leicester City','Man City':'Manchester City','Norwich':'Norwich City','Newcastle':'Newcastle Utd',"Nott'm Forest":"Nottingham Forest"})
bet['Deplasman'] = bet['Deplasman'].replace({'Leeds':'Leeds United','Man United':'Manchester Utd','Leicester':'Leicester City','Man City':'Manchester City','Norwich':'Norwich City','Newcastle':'Newcastle Utd',"Nott'm Forest":"Nottingham Forest"})
bet = bet[["EvSahibi","Deplasman","B365H","B365D","B365A"]]


In [138]:
df =df.merge(bet,on=["EvSahibi","Deplasman"],how='left')




In [139]:
df = df[["Tarih","Maç Haftası","EvSahibi","Deplasman","Ev Gol","Deplasman Gol","Ev Tahmin Edilen Gol","Deplasman Tahmin Edilen Gol","Doğru Skor","Tahmin Edilen Sonuç","Gerçek Durum","Tahmin Edilen MS","B365H","B365D","B365A",'MS EV', 'MS BERABERLIK', 'MS DEPLASMAN',"2.5 Üstü","KG Var"]]

In [140]:
df["B365D"] = round(df['B365D'],2)
df["B365A"] = round(df['B365A'],2)
df["B365H"] = round(df['B365H'],2)

In [141]:
for i,row in df.iterrows():
    sonuc = row["Tahmin Edilen Sonuç"]
    tahmin_edilen = row["Tahmin Edilen MS"]
    e = row["B365H"]
    b = row["B365D"]
    d = row["B365A"]
    if sonuc==0:
        df.at[i,"Kayıp-Kazanç"] = 0
    elif sonuc ==1:
        if tahmin_edilen == "Beraberlik":
            df.at[i,"Kayıp-Kazanç"] = 100 * b
        elif tahmin_edilen == "Ev":
            df.at[i,"Kayıp-Kazanç"] = 100 * e 
        elif tahmin_edilen == "Deplasman":
            df.at[i,"Kayıp-Kazanç"] = 100 * d 

In [142]:
counter = Counter()
counter2 = Counter()
l = []
for i,row in df.iterrows():
    ev_tahmin = row["Ev Tahmin Edilen Gol"]
    dep_tahmin = row["Deplasman Tahmin Edilen Gol"]
    counter[ev_tahmin] +=1
    counter[dep_tahmin] +=1
    ev_gol = row["Ev Gol"]
    dep_gol = row["Deplasman Gol"]
    counter2[ev_gol]+=1
    counter2[dep_gol] +=1


In [143]:
df.to_csv("model.csv")

In [145]:
a = pd.read_csv("model.csv")
a.head()

,Unnamed: 0,Tarih,Maç Haftası,EvSahibi,Deplasman,Ev Gol,Deplasman Gol,Ev Tahmin Edilen Gol,Deplasman Tahmin Edilen Gol,Doğru Skor,...,Tahmin Edilen MS,B365H,B365D,B365A,MS EV,MS BERABERLIK,MS DEPLASMAN,2.5 Üstü,KG Var,Kayıp-Kazanç
0,0,2022-08-05,1,Crystal Palace,Arsenal,0,2,0,1,0,...,Beraberlik,4.20,3.6,1.85,0.237930,0.392068,0.370002,0.376885,0.490934,0.0
1,1,2022-08-06,1,Everton,Chelsea,0,1,1,1,0,...,Beraberlik,5.50,4.0,1.61,0.346602,0.360755,0.292642,0.485171,0.580254,0.0
2,2,2022-08-06,1,Bournemouth,Aston Villa,2,0,1,1,0,...,Beraberlik,3.75,3.5,2.00,0.288145,0.376676,0.335179,0.436808,0.543940,0.0
3,3,2022-08-06,1,Leeds United,Wolves,2,1,1,0,0,...,Ev,2.25,3.4,3.20,0.476487,0.339030,0.184483,0.488990,0.553729,225.0
4,4,2022-08-06,1,Newcastle Utd,Nottingham Forest,2,0,2,0,1,...,Ev,1.66,3.8,5.25,0.674603,0.253745,0.071652,0.531120,0.467888,166.0
